## Data Cleaning

In [ ]:
# imports
import numpy as np
import pandas as pd

In [ ]:
# read data sets
df_imdb = pd.read_csv('IMDB-Movie-Data.csv')
df_ML_id = pd.read_csv('movies.csv')
df_ML_rating = pd.read_csv('ratings.csv')

In [ ]:
# return the index of s in l if there is a match
def checkExist(s, l):
    for i in range (0, len(l)):
        if (s in l[i]):
            return i
    return False

In [ ]:
def dropOld(l):
    for i in range (0, len(l)):
        # get rid of the movies before 2000
        if ('(19' in  l[i]):
            l[i] = np.nan

In [ ]:
# get title of movies, dropping the old ones (not contained in the imdb data frame)
imdb_title = df_imdb['Title']
#df_ML_id = df_ML_id['(19' in df_ML_id.title]
df_ML_id[~df_ML_id['title'].str.contains('\(19')]
df_ML_id = df_ML_id.dropna(axis=0, how='any')

In [ ]:
# insert a new column
col_name = df_imdb.columns.tolist()
col_name.insert(1,'ML_id')  
df_imdb = df_imdb.reindex(columns=col_name)

In [ ]:
# get the columns
df_ML_id = df_ML_id.reset_index(drop=True)
imdb_titles = df_imdb['Title']

In [ ]:
# add the ML id into the imdb dataframe
for index in range (0, len(imdb_titles)):
    id_index = checkExist(imdb_titles[index], df_ML_id['title'])
    if (id_index != False):
        mvid = df_ML_id.loc[id_index:id_index, 'movieId']
        df_imdb.set_value(index, 'ML_id', mvid)

In [ ]:
# drop nan and insert a column for the ML_score
df_imdb = df_imdb.dropna(axis=0, how='any')
col_name = df_imdb.columns.tolist()
col_name.insert(1,'ML_score')  
df_imdb = df_imdb.reindex(columns=col_name)
# add votes column
col_name = df_imdb.columns.tolist()
col_name.insert(1,'ML_votes')  
df_imdb = df_imdb.reindex(columns=col_name)


In [ ]:
# calculate score
def calculateScore(movie_id, index):
    id_rating= df_ML_rating[df_ML_rating['movieId'] == movie_id]
    id_rating = id_rating.reset_index(drop=True)

    df_imdb.set_value(index,'ML_votes',  len(id_rating))
    
    score_sum = id_rating.sum()['rating']
    score = score_sum/len(id_rating['rating'])
    df_imdb.set_value(index, 'ML_score', score*2)
    
    


In [ ]:
df_imdb = df_imdb.reset_index(drop=True)
ML_id = df_imdb['ML_id']
for index in range (0, len(ML_id)):
        calculateScore(ML_id[index], index)

In [ ]:
df_imdb['Overall'] = (df_imdb.ML_score * df_imdb.ML_votes + df_imdb.Rating * df_imdb.Votes) / (df_imdb.Votes+df_imdb.ML_votes)

In [ ]:
df_imdb

In [ ]:
df_imdb.to_csv("integrated_data.csv")